# Análisis exploratorio de datos (EDA)

# 📝 Definición de las Variables

## step: (Numérico, Entero)

- Representa una unidad de tiempo en el mundo real.

- Cada paso (1 step) equivale a 1 hora.

- El dataset cubre un total de 744 steps (30 días).


## type: (Categórico, Texto)

- Tipo de transacción financiera.

- Valores posibles: CASH-IN, CASH-OUT, DEBIT, PAYMENT, TRANSFER.


## amount: (Numérico, Flotante)

- El monto de la transacción en la moneda local.


## nameOrig: (Categórico, Alfanumérico)

- Identificador del cliente que inició la transacción.

- Los clientes que inician con 'C' (ej. C12345) son clientes estándar.

## oldbalanceOrg: (Numérico, Flotante)

- Saldo del cliente de origen antes de la transacción.


## newbalanceOrig: (Numérico, Flotante)

- Saldo del cliente de origen después de la transacción.


## nameDest: (Categórico, Alfanumérico)

- Identificador del cliente receptor (destino) de la transacción.

- Los clientes que inician con 'M' (ej. M12345) son comerciantes (Merchants).


## oldbalanceDest: (Numérico, Flotante)

- Saldo del cliente de destino antes de la transacción.


## newbalanceDest: (Numérico, Flotante)

- Saldo del cliente de destino después de la transacción.


## isFraud: (Binario, Target)

### Esta es tu variable objetivo (target).

- Identifica si una transacción es fraudulenta (1) o legítima (0).

- El fraude en este dataset ocurre exclusivamente en transacciones de tipo TRANSFER y CASH_OUT.


## isFlaggedFraud: (Binario, Flag)

- Una bandera (flag) interna del sistema que marca transacciones sospechosas.

- Se activa cuando se intenta transferir más de 200,000 en una sola transacción.

- Es una regla de negocio simple, no es un resultado de un modelo.

In [1]:
import pandas as pd